# Maastikuökoloogia

__Evelyn Uuemaa, PhD__
__Alexander Kmoch, PhD__

- Praktikumi lisa

Tudengid laevad üles oma ZIP-faili, mis sisaldab maastike rasterfaile (geotifid). Rasterfailidele arvutatakse maastikuindeksid. Tulemuse saab tabelina alla laadida.

Kasutatud materjalid:
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0225734
- https://github.com/martibosch/pylandstats

## Kuidas seda skripti kasutada

Mine hiirega kõige esimesele kastile ja vajuta "Run" nuppu, et üksus läbi jooksutada ja minna järgmiste plokkide juurde. Kui midagi valesti läheb, saad sa lihtsasti uuesti algusest alustada.

In [1]:
from IPython.display import display
import ipywidgets as widgets
import zipfile
import glob
from tqdm import tqdm 

Ülejärgmise ploki juures tuleb sul arvutist valida ZIP-fail, mis sisaldab maastike rasterfaile.

In [2]:
upload = widgets.FileUpload( accept='.zip', multiple=False )

In [3]:
upload

FileUpload(value={}, accept='.zip', description='Upload')

Kui oled faili oma arvutist valinud, peaks üleslaadimise lahter näitama "Upload (1)", mis tähendab, et fail on nüüd valitud.

Jooksuta läbi järgmine plokk. 

In [4]:
t = upload.data[0]

with open('tmp.zip', 'wb') as fh:
    fh.write(t)


with zipfile.ZipFile('tmp.zip', 'r') as zip_ref:
    zip_ref.extractall(".")

all_tifs = glob.glob("*.tif")

# uncomment to see the names of all *.tif files inside the uploaded zip 
# print(all_tifs)

Nüüd on andmed üles laetud ja saame arvutada igale maastikule maastikuindeksid: servatihedus (edge density), eraldiste tihedus (patch density), Shannoni mitmekesisuse indeks (Shannon's diversity metric), kuju indeks (shape index) ja keskmine eraldise suurus (mean patch size).
Tulemused saad allalaadida csv-failina, mille nimi on landscape_metrics.csv. Palun muuda faili nimi pärast allalaadimist nimi sobivaks vastavalt sellele, millise linnu elupaikadele indekseid parasjagu arvutad. Näiteks, "maastikuindeksid_rukkiraak.csv" kui arvutad rukkiräägule.

In [6]:
import pandas as pd
import pylandstats as pls

all_metrics = []

for landscape in tqdm(all_tifs):
    ls = pls.Landscape(landscape)
    metrics_df = ls.compute_landscape_metrics_df(metrics=["edge_density", "patch_density"])
    metrics_df = metrics_df.rename(columns={"edge_density": "ED", "patch_density": "PD"})
    metrics_df["ED"] = ls.compute_landscape_metrics_df(metrics=["edge_density"])
    metrics_df["PD"] = ls.compute_landscape_metrics_df(metrics=["patch_density"])
    metrics_df["MSI"] = ls.shape_index()['shape_index'].mean()
    metrics_df["MPS"] = ls.area(class_val=None, hectares=True)['area'].mean()
    metrics_df.index = [landscape]
    all_metrics.append(metrics_df)
    
all_metrics_df = pd.concat(all_metrics, axis=0)

all_metrics_df.to_csv("landscape_metrics.csv", sep=",", encoding="utf-8")

display(all_metrics_df)

100%|██████████| 50/50 [00:01<00:00, 38.61it/s]


,ED,PD,SDI,MSI,MPS
107946_clip.tif,43.470298,2.219899,1.402347,1.767347,45.047099
79200_clip.tif,28.737318,1.916076,0.835417,1.544709,52.189984
107973_clip.tif,38.761963,3.832153,1.463337,1.362103,26.094992
63408_clip.tif,38.824617,2.854156,1.386288,1.561111,35.036633
46540_clip.tif,41.101047,3.193461,1.088451,1.566708,31.313990
108009_clip.tif,40.957838,3.818800,1.566654,1.374988,26.186233
80662_clip.tif,40.766892,3.193461,1.841722,1.533052,31.313990
123623_clip.tif,31.524262,2.854156,1.222211,1.309402,35.036633
77340_clip.tif,40.766892,3.832153,1.691389,1.376389,26.094992
73708_clip.tif,38.556861,2.834472,1.234223,1.458322,35.279942


[lae tabel alla siit](landscape_metrics.csv)

